In [2]:
!pip install huggingface_hub # a repository where LLMs are stored.

In [3]:
!pip install spacy #It will act as distionary for new words.

In [4]:
!pip install datasets #for the traininig data

In [17]:
!pip install setfit['absa'] #huggingface library consist of some pretrained model,which are not finetuned.

In [1]:
!spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
from setfit import AbsaModel #helps to use some bese models deployed on hugingface which are not finetuned.

In [3]:
model = AbsaModel.from_pretrained('BAAI/bge-small-en-v1.5')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/spacy/cli/_util.py:23: DeprecationWarning: Importing 'parser.split_arg_string' is deprecated, it will only be available in 'shell_completion' in Click 9.0.
  from click.parser import split_arg_string
/usr/local/lib/python3.11/dist-packages/weasel/util/config.py:8: DeprecationWarning: Importing 'parser.split_arg_string' is deprecated, it will only be available in 'shell_completion' in Click 9.0.
  from click.parser import split_arg_string


In [4]:
from datasets import load_dataset

In [5]:
data = load_dataset('tomaarsen/setfit-absa-semeval-restaurants',split='train')

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/147k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/46.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3693 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1134 [00:00<?, ? examples/s]

In [6]:
data.shape

(3693, 4)

In [29]:
train=data.select(range(50)) # training data
test=data.select(range(50,100)) # testing data

In [30]:
train,test

(Dataset({
     features: ['text', 'span', 'label', 'ordinal'],
     num_rows: 50
 }),
 Dataset({
     features: ['text', 'span', 'label', 'ordinal'],
     num_rows: 50
 }))

In [31]:
train[0]

{'text': 'But the staff was so horrible to us.',
 'span': 'staff',
 'label': 'negative',
 'ordinal': 0}

In [32]:
from setfit import AbsaTrainer,TrainingArguments

In [33]:
from transformers import EarlyStoppingCallback

In [34]:
arg=TrainingArguments(
    output_dir='model',
    num_epochs=3,
    batch_size=128,
    eval_strategy='steps',
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True

)

In [35]:
training=AbsaTrainer(
    model,
    args=arg,
    train_dataset=train,
    eval_dataset=test,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]

)

Map:   0%|          | 0/76 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Now that you have created the `AbsaTrainer` object, you can start the training process by calling the `train()` method on the `trainer` object.

In [36]:
training.train()

***** Running training *****
  Num unique pairs = 3252
  Batch size = 128
  Num epochs = 3


Step,Training Loss,Validation Loss
50,0.001600,0.293528


TypeError: Wrong key type: '27' of type '<class 'numpy.int64'>'. Expected one of int, slice, range, str or Iterable.

In [ ]:
training.evaluate(test)

In [ ]:
res=model.predict("Best pizza outside of Italy and really tasty.")
res